# Proyecto 1.1 BI 

Integrantes:
-Maria Fernanda De La Hoz
-Silvana Sandoval 
-Gabriela Soler 

## Preprocesamiento y análisis de datos

### Instalación de spaCy y Modelos de lenguaje

In [1]:
!pip install spacy
!python -m spacy download es_core_news_md


     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     --------------------------------------- 0.0/42.3 MB 217.9 kB/s eta 0:03:14
     --------------------------------------- 0.0/42.3 MB 326.8 kB/s eta 0:02:10
     --------------------------------------- 0.1/42.3 MB 438.9 kB/s eta 0:01:37
     ---------------------------------------- 0.5/42.3 MB 2.4 MB/s eta 0:00:18
     - -------------------------------------- 1.1/42.3 MB 4.5 MB/s eta 0:00:10
     - -------------------------------------- 1.3/42.3 MB 4.7 MB/s eta 0:00:09
     - -------------------------------------- 1.3/42.3 MB 4.7 MB/s eta 0:00:09
     - -------------------------------------- 1.3/42.3 MB 3.7 MB/s eta 0:00:12
     -- ------------------------------------- 2.4/42.3 MB 5.5 MB/s eta 0:00:08
     --- ------------------------------------ 3.4/42.3 MB 7.0 MB/s eta 0:00:06
     ---- ----------------------------------- 4.4/42.3 MB 8.4 MB/s eta 0:00:05
     ----- ---------------------------------- 5.4/42.3 M

### Importación de las librerias pertinentes

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix

### Carga del modelo de lenguaje

In [2]:
import spacy
nlp = spacy.load("es_core_news_md") #En este caso cargamos el modelo de lenguaje español

### Lectura de datos 

In [3]:
import pandas as pd

data = pd.read_excel("ODScat_345.xlsx")

,Textos_espanol,sdg
0,"Por ejemplo, el nÃºmero de consultas externas ...",3
1,"En 2007, el gobierno central financió directam...",3
2,"Claramente, hay muchos otros factores en juego...",3
3,"Por ejemplo, el estado australiano de Victoria...",3
4,"El consumo anual de alcohol se estima en 15,7 ...",3


## Entendimiento de los datos

In [15]:
print(data.head(5))
print("Total de registros: ", data.shape[0])

                                      Textos_espanol  sdg
0  Por ejemplo, el nÃºmero de consultas externas ...    3
1  En 2007, el gobierno central financió directam...    3
2  Claramente, hay muchos otros factores en juego...    3
3  Por ejemplo, el estado australiano de Victoria...    3
4  El consumo anual de alcohol se estima en 15,7 ...    3
Total de registros:  4049


Se tienen 4049 opiniones de ciudadanos y hay 2 variables: la primera corresponde a la variable Textos_espanol, que es una cadena de caracteres y es la opinión de los ciudadanos. La segunda variable es sdg, que es la variable objetivo, es decir, el ODS correspondiente.

### Unicidad

In [16]:
print("Duplicados: ", data.duplicated().sum())

Duplicados:  0


### Completitud

In [17]:
print(data.isnull().sum())

Textos_espanol    0
sdg               0
dtype: int64


### Validez

In [18]:
data.describe()

,sdg
count,4049.000000
mean,4.051124
std,0.814338
min,3.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [19]:
data.dtypes

Textos_espanol    object
sdg                int64
dtype: object

## Preprocesamiento del texto

In [4]:
# Aplicar spaCy a la columna de opiniones en español
dataCopy = data.copy()
dataCopy['procesado'] = dataCopy['Textos_espanol'].apply(lambda x: nlp(x))
dataCopy[['Textos_espanol', 'procesado']].head()

,Textos_espanol,procesado
0,"Por ejemplo, el nÃºmero de consultas externas ...","(Por, ejemplo, ,, el, nÃºmero, de, consultas, ..."
1,"En 2007, el gobierno central financió directam...","(En, 2007, ,, el, gobierno, central, financió,..."
2,"Claramente, hay muchos otros factores en juego...","(Claramente, ,, hay, muchos, otros, factores, ..."
3,"Por ejemplo, el estado australiano de Victoria...","(Por, ejemplo, ,, el, estado, australiano, de,..."
4,"El consumo anual de alcohol se estima en 15,7 ...","(El, consumo, anual, de, alcohol, se, estima, ..."


### Tokenización

In [5]:
dataCopy['tokens'] = dataCopy['procesado'].apply(lambda doc: [token.text for token in doc])
dataCopy[['Textos_espanol', 'tokens']].head()

,Textos_espanol,tokens
0,"Por ejemplo, el nÃºmero de consultas externas ...","[Por, ejemplo, ,, el, nÃºmero, de, consultas, ..."
1,"En 2007, el gobierno central financió directam...","[En, 2007, ,, el, gobierno, central, financió,..."
2,"Claramente, hay muchos otros factores en juego...","[Claramente, ,, hay, muchos, otros, factores, ..."
3,"Por ejemplo, el estado australiano de Victoria...","[Por, ejemplo, ,, el, estado, australiano, de,..."
4,"El consumo anual de alcohol se estima en 15,7 ...","[El, consumo, anual, de, alcohol, se, estima, ..."


### Lematización-Obtener la forma base de las palabras

In [6]:
dataCopy['lemmas'] = dataCopy['procesado'].apply(lambda doc: [token.lemma_ for token in doc])
dataCopy[['Textos_espanol', 'lemmas']].head()

,Textos_espanol,lemmas
0,"Por ejemplo, el nÃºmero de consultas externas ...","[por, ejemplo, ,, el, nãºmero, de, consulta, e..."
1,"En 2007, el gobierno central financió directam...","[en, 2007, ,, el, gobierno, central, financiar..."
2,"Claramente, hay muchos otros factores en juego...","[claramente, ,, haber, mucho, otro, factor, en..."
3,"Por ejemplo, el estado australiano de Victoria...","[por, ejemplo, ,, el, estado, australiano, de,..."
4,"El consumo anual de alcohol se estima en 15,7 ...","[el, consumo, anual, de, alcohol, él, estimar,..."


### Eliminación de stopWords

In [8]:
dataCopy['lemmas_sin_stopwords'] = dataCopy['procesado'].apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop])
dataCopy[['Textos_espanol', 'lemmas_sin_stopwords']].head()

,Textos_espanol,lemmas_sin_stopwords
0,"Por ejemplo, el nÃºmero de consultas externas ...","[ejemplo, ,, nãºmero, consulta, externo, espec..."
1,"En 2007, el gobierno central financió directam...","[2007, ,, gobierno, central, financiar, direct..."
2,"Claramente, hay muchos otros factores en juego...","[claramente, ,, factor, juego, ,, particular, ..."
3,"Por ejemplo, el estado australiano de Victoria...","[ejemplo, ,, australiano, Victoria, crear, age..."
4,"El consumo anual de alcohol se estima en 15,7 ...","[consumo, anual, alcohol, estimar, 15.7, litro..."


### Filtrado de caracteres especiales

In [9]:
dataCopy['lemmas_limpios'] = dataCopy['lemmas_sin_stopwords'].apply(lambda lemmas: [lemma for lemma in lemmas if lemma.isalpha()])
dataCopy[['Textos_espanol', 'lemmas_limpios']].head()

,Textos_espanol,lemmas_limpios
0,"Por ejemplo, el nÃºmero de consultas externas ...","[ejemplo, nãºmero, consulta, externo, especial..."
1,"En 2007, el gobierno central financió directam...","[gobierno, central, financiar, directamente, g..."
2,"Claramente, hay muchos otros factores en juego...","[claramente, factor, juego, particular, varied..."
3,"Por ejemplo, el estado australiano de Victoria...","[ejemplo, australiano, Victoria, crear, agenci..."
4,"El consumo anual de alcohol se estima en 15,7 ...","[consumo, anual, alcohol, estimar, litro, año,..."


### Entidades resultantes

In [10]:
dataCopy['entidades'] = dataCopy['procesado'].apply(lambda doc: [(ent.text, ent.label_) for ent in doc.ents])
dataCopy[['Textos_espanol', 'entidades']].head()

,Textos_espanol,entidades
0,"Por ejemplo, el nÃºmero de consultas externas ...","[(nÃºmero, MISC), (SP, ORG), (comparaciÃ³n, LO..."
1,"En 2007, el gobierno central financió directam...","[(La mayor parte, MISC), (El gobierno provinci..."
2,"Claramente, hay muchos otros factores en juego...","[(Por ejemplo, MISC), (Por lo tanto, MISC), (L..."
3,"Por ejemplo, el estado australiano de Victoria...","[(Victoria, LOC), (EE.UU., LOC), (California, ..."
4,"El consumo anual de alcohol se estima en 15,7 ...","[(OCDE, ORG), (OMS, ORG), (Si bien la prevalen..."


### Clasificación de palabras

In [11]:
dataCopy['pos_tags'] = dataCopy['procesado'].apply(lambda doc: [(token.text, token.pos_) for token in doc])
dataCopy[['Textos_espanol', 'pos_tags']].head()

,Textos_espanol,pos_tags
0,"Por ejemplo, el nÃºmero de consultas externas ...","[(Por, ADP), (ejemplo, NOUN), (,, PUNCT), (el,..."
1,"En 2007, el gobierno central financió directam...","[(En, ADP), (2007, NOUN), (,, PUNCT), (el, DET..."
2,"Claramente, hay muchos otros factores en juego...","[(Claramente, ADV), (,, PUNCT), (hay, AUX), (m..."
3,"Por ejemplo, el estado australiano de Victoria...","[(Por, ADP), (ejemplo, NOUN), (,, PUNCT), (el,..."
4,"El consumo anual de alcohol se estima en 15,7 ...","[(El, DET), (consumo, NOUN), (anual, ADJ), (de..."


### Texto preprocesado Final

In [13]:
dataCopy['texto_preprocesado'] = dataCopy['lemmas_limpios'].apply(lambda lemmas: ' '.join(lemmas))
dataCopy[['Textos_espanol', 'texto_preprocesado']].head()

,Textos_espanol,texto_preprocesado
0,"Por ejemplo, el nÃºmero de consultas externas ...",ejemplo nãºmero consulta externo especialista ...
1,"En 2007, el gobierno central financió directam...",gobierno central financiar directamente gasto ...
2,"Claramente, hay muchos otros factores en juego...",claramente factor juego particular variedad ba...
3,"Por ejemplo, el estado australiano de Victoria...",ejemplo australiano Victoria crear agencia pro...
4,"El consumo anual de alcohol se estima en 15,7 ...",consumo anual alcohol estimar litro año hombre...


### Guardar datos

In [14]:

dataCopy.to_excel("datos_preprocesados.xlsx", index=False)
print("Los datos preprocesados fueron cargados exitosamente")


Los datos preprocesados fueron cargados exitosamente


## Transformación

### 1. Term Frequency Inverse Document Frequency

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(dataCopy['texto_preprocesado']).toarray()

### 2. Count Vectorizer

In [24]:
count = CountVectorizer()
X_count = count.fit_transform(dataCopy['texto_preprocesado'])
print(X_count.shape)
X_count.toarray()[0]

(4049, 12420)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### 3. Count Vectorizer Binario

In [25]:
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(dataCopy['texto_preprocesado'])
print(X_dummy.shape)
X_dummy.toarray()[0]

(4049, 12420)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## Búsqueda de hiperparámetros para el Árbol de Decisión

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=0)

Los siguientes parámetros se usaran para realizar la busqueda de los mejores hiperparámetros para este conjunto de datos.

In [ ]:
param_grid = {'criterion': ['entropy', 'gini'],
              'max_depth': [2, 4, 6, 8],
              'min_samples_split': [2, 3, 5, 10]}

Además, para asegurarnos de escoger la mejor opción posible, usaremos un objeto KFold() para realizar validación cruzada con 15 subconjuntos de los datos

In [ ]:
kfold = KFold(n_splits=15, shuffle= True, random_state=0)

Por último, haremos la búsqueda de los mejores parametros con un objeto GridSearchCV(). Esta se realizará con un n_jobs = -1 ya que deseamos que todos los procesadores sean usados para la busqueda de los hiperparámetros.

In [ ]:
grid = GridSearchCV(decision_tree, param_grid, cv=kfold, n_jobs=-1)

Ejecutaremos la búsqueda de los hiperparámetros

In [ ]:
grid.fit(x_train, y_train)
#Revisar si tenemos x_train y y_train. Al no tenerlos, preguntar si vamos a usar una pipeline para tener los datos listos

Se obtendran los mejores valores para los hiperparámetros

In [ ]:
print("Mejores parámetros: {}".format(grid.best_params_))

Se obtendrá el mejor modelo para el Árbol de Decisión

In [ ]:
best_tree = grid.best_estimator_

## Busqueda de los mejores hiperparámetros para Naïve Bayes

## Busqueda de los mejores hiperparámetros para K-Vecinos